<a href="https://colab.research.google.com/github/Papa-Panda/Paper_reading/blob/main/two_stream.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
# https://www.bilibili.com/video/BV1mq4y1x7RU/?spm_id_from=333.337.search-card.all.click&vd_source=83baba81780fd95e96c22e9346057527

In [2]:
import torch
import torch.nn as nn
import torchvision.models as models

# Define the spatial stream model
class SpatialStreamCNN(nn.Module):
    def __init__(self, num_classes=101):
        super(SpatialStreamCNN, self).__init__()
        # Load a pre-trained ResNet model
        self.spatial_model = models.resnet50(pretrained=True)
        # Replace the final layer to adapt to the number of action classes
        self.spatial_model.fc = nn.Linear(self.spatial_model.fc.in_features, num_classes)

    def forward(self, x):
        return self.spatial_model(x)


In [3]:
class TemporalStreamCNN(nn.Module):
    def __init__(self, num_classes=101):
        super(TemporalStreamCNN, self).__init__()
        # Load a pre-trained ResNet model
        self.temporal_model = models.resnet50(pretrained=True)
        # Optical flow has 2 channels (for x and y flow)
        self.temporal_model.conv1 = nn.Conv2d(2, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
        # Replace the final layer to adapt to the number of action classes
        self.temporal_model.fc = nn.Linear(self.temporal_model.fc.in_features, num_classes)

    def forward(self, x):
        return self.temporal_model(x)


In [4]:
class TwoStreamActionRecognition(nn.Module):
    def __init__(self, num_classes=101):
        super(TwoStreamActionRecognition, self).__init__()
        self.spatial_stream = SpatialStreamCNN(num_classes=num_classes)
        self.temporal_stream = TemporalStreamCNN(num_classes=num_classes)

    def forward(self, rgb_frames, optical_flow_frames):
        spatial_out = self.spatial_stream(rgb_frames)   # Spatial stream output
        temporal_out = self.temporal_stream(optical_flow_frames)  # Temporal stream output

        # Combine outputs (you can also use other strategies like weighted sum)
        combined_out = (spatial_out + temporal_out) / 2

        return combined_out


In [6]:
# Config

num_epochs = 100

In [7]:
# Sample training loop
model = TwoStreamActionRecognition(num_classes=101)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
criterion = nn.CrossEntropyLoss()

for epoch in range(num_epochs):
    for rgb_frames, optical_flow_frames, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(rgb_frames, optical_flow_frames)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()


NameError: name 'train_loader' is not defined

In [8]:
import cv2

def compute_optical_flow(prev_frame, next_frame):
    prev_gray = cv2.cvtColor(prev_frame, cv2.COLOR_RGB2GRAY)
    next_gray = cv2.cvtColor(next_frame, cv2.COLOR_RGB2GRAY)
    flow = cv2.calcOpticalFlowFarneback(prev_gray, next_gray, None, 0.5, 3, 15, 3, 5, 1.2, 0)
    return flow

In [9]:
# Testing on a single video
rgb_frames = extract_rgb_frames(video)  # Extract frames from the video
optical_flow_frames = compute_optical_flow_frames(rgb_frames)  # Compute optical flow
output = model(rgb_frames, optical_flow_frames)
predicted_class = torch.argmax(output, dim=1)


NameError: name 'extract_rgb_frames' is not defined